In [ ]:
print('Hello World. Lets start our journey towards Data Scien & AI')

Hello World. Lets start our journey towards Data Scien & AI


In [ ]:
# Import the  library
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Phase - 1

### Accessing the Downloaded .csv Dataset File

In [ ]:
df = pd.read_csv('online_retail.csv')

In [ ]:
# Step 1: Install the KaggleHub library
# The '!' command runs a shell command in Google Colab/Jupyter.
!pip install kagglehub

# Step 2: Import the necessary libraries
import kagglehub

import os # This library helps work with file paths

### The following method ensures that you will always work with the latest version of the dataset.

In [ ]:
# Step 3: Download the dataset
# This will download the files to a local folder and return the path.
# In Google Colab, it will ask for authentication the first time.
print("Downloading the dataset...")

In [ ]:
# Tells the kagglehub library to find the dataset identified by "pavansubhasht/ibm-hr-analytics-attrition-dataset"
# Saves the files to a temporary folder on the computer and then stores the location (the path) of that folder in the variable called 'path'
path = kagglehub.dataset_download("ulrikthygepedersen/online-retail-dataset")
print(f"Dataset downloaded to: {path}")

Dataset downloaded to: C:\Users\manwarsadat\.cache\kagglehub\datasets\ulrikthygepedersen\online-retail-dataset\versions\2


In [ ]:
file_name = 'online_retail.csv'

In [ ]:
df = pd.read_csv(file_path, encoding='latin1')

In [ ]:
# Step 5: Display the first 5 rows to confirm it's working
print("\nDataset loaded successfully!")
print(df.head())


Dataset loaded successfully!
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

           InvoiceDate  UnitPrice  CustomerID         Country  
0  2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1  2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2  2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3  2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4  2010-12-01 08:26:00       3.39     17850.0  United Kingdom  


# Phase - 2 

### Pre-processing (dataset clean-up)

In [ ]:
#lets check if we have null values in the dataset

print(df.isnull().sum())


In [ ]:
# Remove rows with missing CustomerID
df = df.dropna(subset=['CustomerID'])

# Remove cancelled orders (InvoiceNo starts with 'C')
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

# Remove negative Quantity and UnitPrice
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# Add TotalPrice column
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Convert InvoiceDate to datetime type
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

print("Data preprocessing complete. Data sample:")
print(df.head())
print(df.shape)

## Exploratory Data Analysis (EDA) 
#### why we used InoviceDate as index values?
#### using InvoiceDate as the index enables us to cluster transactions across all customers within specific time windows, which is essential for Exploratory Data Analysis and understanding sales using time or changes in customer behavior over time.


In [ ]:

# InvoiceDate as index
df.set_index('InvoiceDate', inplace=True)

# Monthly revenue trend
df.resample('M')['TotalPrice'].sum().plot(figsize=(10,6), title='Monthly Revenue over Time')
plt.ylabel('Revenue')
plt.show()

# Top 10 countries by revenue
df_reset = df.reset_index()
country_sales = df_reset.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False).head(10)
country_sales.plot(kind='bar', figsize=(10,5), title='Top 10 Countries by Revenue')
plt.ylabel('Revenue')
plt.show()

# Top customers by total spend
customer_sales = df_reset.groupby('CustomerID')['TotalPrice'].sum().sort_values(ascending=False).head(10)
customer_sales.plot(kind='bar', figsize=(10,5), title='Top Customers by Total Spend')
plt.ylabel('Total Spend')
plt.show()

## Feature Engineering

In [ ]:
# Calculate latest date plus one day for Recency calculation
latest_date = df.index.max() + pd.Timedelta(days=1)

# Group by CustomerID and calculate RFM metrics
grouped = df_reset.groupby('CustomerID')
rfm = grouped.agg({
    'InvoiceDate': lambda x: (latest_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalPrice': 'sum'
})
rfm.rename(columns={'InvoiceDate': 'Recency', 'InvoiceNo': 'Frequency', 'TotalPrice': 'Monetary'}, inplace=True)

# Behavioral features
extra = grouped.agg({
    'Quantity': ['sum', 'mean'],
    'TotalPrice': 'mean',
    'StockCode': 'nunique'
})
extra.columns = ['TotalQuantity', 'AvgQuantity', 'AvgSpend', 'ProductDiversity']

# Merge behavioral features with RFM
df_rfm = rfm.merge(extra, on='CustomerID')
print(df_rfm.head())


## The RFM model explained
#### The RFM model is based on the idea that a customer's past behavior is a good predictor of their future behavior. The three components are: 
#### Recency: Measures how recently a customer made a purchase. The more recent the transaction, the more likely the customer is to respond to future promotions.
#### Frequency: Measures how often a customer makes a purchase within a given time period. Frequent buyers are generally more engaged and loyal.
#### Monetary Value: Measures how much a customer spends. High-spending customers are more valuable and should be treated accordingly. 

### RFM Scoring with quantiles 

In [ ]:

df_rfm['R_Score'] = pd.qcut(df_rfm['Recency'], 5, labels=[5,4,3,2,1])
df_rfm['F_Score'] = pd.qcut(df_rfm['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
df_rfm['M_Score'] = pd.qcut(df_rfm['Monetary'], 5, labels=[1,2,3,4,5])

# Compute combined RFM score
df_rfm['RFM_Score'] = df_rfm[['R_Score', 'F_Score', 'M_Score']].astype(int).sum(axis=1)

print(df_rfm[['Recency', 'Frequency', 'Monetary', 'R_Score', 'F_Score', 'M_Score', 'RFM_Score']].head())


